# Investments I: Fundamentals of Performance Evaluation
by Scott Weisbenner University of Illinois at Urbana-Champaign
## Module 1-7

In [57]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import numpy.testing as npt
import scipy.optimize as opt

def fV_p(w, rho, s):
    """
    Portfolio variance
    :param w: Allocation weights
    :param rho: Asset correlations
    :param s: Variance of asset return
    """
    cov=s@s.T * rho
#     print(f'cov\n{cov}\nw\n{w}')
    V=np.sum(np.dot(cov, w) * w, axis=0)
#     print(f'V\n{np.reshape(V, (1,-1))}')
    return np.reshape(V, (1,-1))

def fr_p(w, r):
    """
    Portfolio return
    :param w: Allocation weights
    :param r: Asset returns
    """
    return np.dot(r.T, w)

def fsharpe(w, r, rho, s):
    """Sharpe ratio"""
    s2=fV_p(w, rho, s)[0,0]
    E=fr_p(w, r)[0,0]
    S=(E-rf)/np.sqrt(s2)
    return S

def print_portfolio(w):
    print(f'Portfolio weights ' + ', '.join(f'{x:.3f}' for x in w[:,0]))
    s2=fV_p(w, rho, s)[0,0]
    print(f'Portfolio variance {s2:.3f}, std. dev. {np.sqrt(s2):.3f}')
    print(f'Return {fr_p(w, r)[0,0]:.3f}')
    S=fsharpe(w, r, rho, s)
    print(f'Sharpe ratio {S:.3f}')

In [58]:
r=np.asfarray([[0.112, 0.19, 0.175, 0.111]]).T  # Large, small, value, growth returns
s=np.asfarray([[0.192, 0.394, 0.334, 0.225]]).T  # Large, small, value, growth std. dev. in returns
rho=[
    [1, 0.69, 0.8, 0.94],
    [0.69, 1, 0.84, 0.65],
    [0.8, 0.84, 1, 0.7],
    [0.94, 0.65, 0.7, 1]
]
rf=0.035  # Risk-free return rate

### Q1: Return of Porfolio with Fixed Asset Allocation

In [59]:
w=np.asfarray([[0.25, 0.25, 0.25, 0.25]]).T

print_portfolio(w)

s2=fV_p(w, rho, s)[0,0]

Portfolio weights 0.250, 0.250, 0.250, 0.250
Portfolio variance 0.068, std. dev. 0.260
Return 0.147
Sharpe ratio 0.430


### Q1-2: Portfolio with Same Variance as Fixed Allocation having Maximum Return
#### Negative weights *not* allowed

In [60]:
# Note, reuses s2 from previous block
def f(x):
    x=np.reshape(x, (-1,1))
    return -fr_p(x, r)[0,0]
x0=[0.25,0.25,0.25,0.25]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
    {'type':'eq', 'fun':lambda x: fV_p(x, rho, s)[0,0]-s2}  # Fixed variance
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x
    pass

result=opt.minimize(f, x0, method='SLSQP', constraints=c)
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)
npt.assert_allclose(s2, fV_p(w, rho, s)[0,0], rtol=1e-4)
assert fr_p(w, r)[0,0] >= r_p

print_portfolio(w)

Portfolio weights 0.459, 0.205, 0.336, -0.000
Portfolio variance 0.068, std. dev. 0.260
Return 0.149
Sharpe ratio 0.439


### Q2: Portfolio with Same Variance as Fixed Allocation having Maximum Return
#### Negative weights allowed

In [61]:
# Note, reuses s2 from previous block
def f(x):
    x=np.reshape(x, (-1,1))
    return -fr_p(x, r)[0,0]
x0=[1,1,1,1]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
    {'type':'eq', 'fun':lambda x: fV_p(x, rho, s)[0,0]-s2}  # Fixed variance
)
result=opt.minimize(f, x0, method='SLSQP', constraints=c)
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)
npt.assert_allclose(s2, fV_p(w, rho, s)[0,0], rtol=1e-4)
assert fr_p(w, r)[0,0] >= r_p

print_portfolio(w)

Portfolio weights 1.125, 0.290, 0.252, -0.667
Portfolio variance 0.068, std. dev. 0.260
Return 0.151
Sharpe ratio 0.446


### Q3: Portfolio with Minimum Variance
#### Negative weights *not* allowed

In [62]:
# Setup
def f(x):
    x=np.reshape(x, (-1,1))
    return fV_p(x, rho, s)[0,0]
x0=[0.25, 0.25, 0.25, 0.25]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

# Optimize
result=opt.minimize(f, x0, method='SLSQP', constraints=c)
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)

print_portfolio(w)

Portfolio weights 1.000, 0.000, -0.000, 0.000
Portfolio variance 0.037, std. dev. 0.192
Return 0.112
Sharpe ratio 0.401


### Q4: Portfolio with Maximum Sharpe Ratio
#### Negative weights *not* allowed

In [63]:
# Setup
def f(x):
    x=np.reshape(x, (-1,1))
    return -fsharpe(x, r, rho, s)  # Maximize
x0=[1, 1, 1, 1]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

# Optimize
result=opt.minimize(f, x0, method='SLSQP', constraints=c)
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)

print_portfolio(w)

Portfolio weights 0.523, 0.188, 0.289, 0.000
Portfolio variance 0.063, std. dev. 0.250
Return 0.145
Sharpe ratio 0.439
